In [ ]:
pro_key = 'YOUR-KEY-FOR-GIGACHAT-PRO-HERE'
key = 'YOUR-KEY-HERE'

In [ ]:
!pip install 'pyqt5<5.16' 'pyqtwebengine<5.16' 'typing-extensions==4.5.0'

In [ ]:
!pip install typing torch transformers tokenizers openpyxl xlsxwriter

In [ ]:
!pip install langchain
!pip install gigachain-community gigachain-cli
!gigachain install-rus-certs

In [6]:
import pandas as pd
import numpy as np
import torch
import tqdm
import re
from transformers import BertModel, BertTokenizer, AutoTokenizer, AutoModel
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat
from sklearn.metrics.pairwise import cosine_similarity

# Baseline

In [21]:
system_message = '''
Ты - профессиональный программист и учитель.
Я пришлю тебе условие задачи, верное решение и мое решение с ошибками. 
Дай крайне короткий ответ в одном предложении об ошибке в решении.
Ошибки могут быть синтаксические и логические.
Объясни ошибку через смысл задания. Ни в коем случае не ссылайся на реализацию в верном решении.
Ни в коем случае не присылай исправленный код решения.
Используй следующие фразы: "ваш код", "выполняет условия", "попробуйте изменить", "условия задания", "некорректно выполняет", "скорректировать ошибку" или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде. 
Обращайся ко мне лично и на вы.
'''


prev_prompt = '''
Ты - профессиональный программист и учитель. 
Я пришлю тебе условие задачи, эталонное решение и мое решение с ошибками. 
Дай очень короткий ответ в одном предложении о синтаксических и логических ошибках в коде моего решения, если они есть.
Учти, что синтаксические ошибки могут быть связаны с кавычками: ' и ".
Объясни ошибку через упоминание смысла задания, ни в коем случае не упоминай, как сделано в эталонном решении. 
Ни в коем случае не присылай исправленный код моего решения, вообще не присылай код. 
Пиши "обратите внимание," или задавай наводящие вопросы, если мне нужно обратить внимание на ошибку в моем коде. 
Обращайся ко мне лично и на вы.
'''

In [8]:
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [22]:
def predict(messages, tasks: pd.DataFrame, row_solution: pd.Series, synt_errors: str) -> str:
    text = f'''
    Условие задачи:
    {tasks[tasks['id'] == row_solution["task_id"]]['description'].values[0]}
    
    Мое решение с ошибкой: 
    {row_solution["student_solution"]}
    
    Синтаксические ошибки в решении:
    {synt_errors}
    
    Верное решение:
    {tasks[tasks['id'] == row_solution["task_id"]]['author_solution'].values[0]}

    Пример, как могут выглядеть твои ответы:
    1. Обратите внимание на неверный оператор сравнения — необходимо проверить, что цвет не находится в списке cite_project.
    2. Обратите внимание, что после else не нужно прописывать никаких условий.
    3. Вы не добавили переменные, которые функция будет принимать как аргументы.
    
    Используй только эти фразы: "ваш код", "выполняет условия", "попробуйте изменить", "условия задания", "некорректно выполняет", "скорректировать ошибку" или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде. 
    '''
    
    messages.append(HumanMessage(content=text))
    res = chat(messages)
    messages.append(res)
        
    return res.content

def get_sentence_embedding(tokenizer, sentence: str) -> torch.Tensor:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding

def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])

def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])

def remove_after_substring(text, substring='исправлен'):    
    index = text.lower().find(substring.lower())
    if index != -1:
        return text[:index]
    return text

def remove_code_from_text(text):
    code_pattern = re.compile(r'\b(def|class|if|else|elif|for|while|try|except|finally|with|import|from|print|return|break|continue|pass|raise|assert|yield|async|await|lambda|global|nonlocal|del|exec|eval)\b.*', re.DOTALL)
    fenced_code_pattern = re.compile(r'```.*?```', re.DOTALL)
    cleaned_text = code_pattern.sub('', text)
    cleaned_text = fenced_code_pattern.sub('', cleaned_text)
    return cleaned_text

def generate_submit(test_solutions_path: str, test_tasks_path: str, 
                    test_tests_path: str, syntacsis_erors_path:str, predict_func, 
                    save_path: str, use_tqdm: bool = True) -> None:
    test_solutions = pd.read_excel(test_solutions_path)
    test_tasks = pd.read_excel(test_tasks_path)
    test_tests = pd.read_excel(test_tests_path)
    with_syntacsis = pd.read_csv(syntacsis_erors_path)
    synt_errors_df = with_syntacsis[~with_syntacsis['message'].isna()]

    bar = range(len(test_solutions))
    if use_tqdm:
        bar = tqdm.tqdm(bar, desc="Predicting")

    submit_df = pd.DataFrame(columns=["solution_id", "author_comment", "author_comment_embedding"])
    for i in bar:
        idx = test_solutions.index[i]
        solution_row = test_solutions.iloc[i]
        
        if i in synt_errors_df.index:
            synt_errors = synt_errors_df.loc[i]['message']
        else:
            synt_errors = 'не были найдены'
        
        try:
            chat = GigaChat(credentials=pro_key, model='GigaChat-Pro', verify_ssl_certs=False)
            messages = [
                SystemMessage(
                    content=system_message
                )
            ]
            text = predict_func(messages, test_tasks, solution_row, synt_errors)
            if 'исправлен' in text.lower():
                text = remove_after_substring(text)
            else:
                text = remove_code_from_text(text)
            
            embedding = embedding2string(get_sentence_embedding(tokenizer, text))
            submit_df.loc[i] = [idx, text, embedding]
        # если закончились токены
        except:
            submit_df.to_csv(save_path, index=False)
            return i, submit_df
        submit_df.to_csv(save_path, index=False)

In [23]:
generate_submit(
    test_solutions_path="/kaggle/input/ai-assistant-data/solutions_test.xlsx",
    test_tasks_path="/kaggle/input/ai-assistant-data/tasks_test.xlsx",
    test_tests_path="/kaggle/input/ai-assistant-data/tests_test.xlsx",
    syntacsis_erors_path="/kaggle/input/ai-assistant-data/test_solutions_with_pyright.csv",
    predict_func=predict,
    save_path="submission_gigachat_wpop_words.csv",
    use_tqdm=True,
)

Predicting: 100%|██████████| 3/3 [00:17<00:00,  5.81s/it]


## Check

In [24]:
curr_submission = pd.read_csv('/kaggle/working/submission_gigachat_wpop_words.csv')
test_solutions = pd.read_excel("/kaggle/input/ai-assistant-data/solutions_test.xlsx")
with_syntacsis = pd.read_csv("/kaggle/input/ai-assistant-data/test_solutions_with_pyright.csv")

In [25]:
cnt = 0
for sol, error, comment in zip(
    test_solutions['student_solution'], 
    with_syntacsis['message'],
    curr_submission['author_comment']
):
    if cnt == 6:
        break
    print(sol)
    print(error)
    print(comment)
    print('_'*50)
    cnt += 1

discount  = float(input())
money = int(input())

#ваш код ниже

print('Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.')
nan
Ваше решение некорректно выполняет условия задания, так как использует обычную строку вместо f-строки для форматирования вывода. Попробуйте изменить код, чтобы использовать f-строку во втором примере вывода.
__________________________________________________
discount  = float(input())
money = int(input())

#ваш код ниже

print(f'Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.)
"(" was not closed
Ваше решение некорректно выполняет задачу, так как не учитывает тип данных переменной money. Переменная money является целым числом, поэтому при вычитании вещественного числа (скидка) результат также должен быть вещественным. Необходимо привести тип данных к float перед выполнением операции вычитания.
___________

## For solutions with syntax errors, look for similar answers in train using bert, replace gigachat answers with them

In [28]:
submission = pd.read_csv("/kaggle/input/ai-assistant-data/submission_gigachat_v11.csv")
train_solutions = pd.read_csv("/kaggle/input/ai-assistant-data/train_solutions_with_pyright.csv")
pyright_solutions = pd.read_csv("/kaggle/input/ai-assistant-data/test_solutions_with_pyright.csv")
submission = pd.merge(submission, pyright_solutions[["id", "message"]], left_on="solution_id", right_on="id", how="left").drop("id", axis=1)

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# Load AutoModel from huggingface model repository
tokenizer_new = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model_new = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

# Function to compute embeddings for a text
def compute_embeddings(text):
    # Tokenize sentences
    encoded_input = tokenizer_new(text, padding=True, truncation=True, max_length=24, return_tensors='pt')
    
    # Compute token embeddings
    with torch.no_grad():
        model_output = model_new(**encoded_input)

    # Perform pooling. In this case, mean pooling
    return mean_pooling(model_output, encoded_input['attention_mask']).numpy().squeeze()

# Apply function to the text column and create a new embeddings column
train_solutions['good_embedding'] = train_solutions['author_comment'].apply(compute_embeddings)

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [29]:
submission["good_embedding"] = submission["author_comment"].apply(compute_embeddings)

In [30]:
# Function to replace texts based on cosine similarity
def replace_text_based_on_similarity(train, submission):
    train = train.copy()
    submission = submission.copy()
    # train['author_comment_embedding'] = train['author_comment_embedding'].apply(lambda x: convert_embedding(x))
    # submission['author_comment_embedding'] = submission['author_comment_embedding'].apply(lambda x: convert_embedding(x))
    # Loop over rows in submission where true_false is True
    for idx, row in submission[submission['message'].notna()].iterrows():
        # Calculate cosine similarities between the current row's embedding and all true text embeddings
        similarities = cosine_similarity([row['good_embedding']], train[train['message'].notna()]['good_embedding'].tolist())
        
        # Find the index of the most similar embedding in train
        most_similar_idx = np.argmax(similarities)
        
        # Replace the text in submission with the most similar true_text from train
        submission.at[idx, 'author_comment'] = train.at[most_similar_idx, "author_comment"]

    return submission

# Example usage
df2_updated = replace_text_based_on_similarity(train_solutions, submission)

# Now df2 will have the most similar true_text from df1 where true_false is True
df2_updated["author_comment_embedding"] = df2_updated["author_comment"].apply(lambda x: embedding2string(get_sentence_embedding(tokenizer, x)))
df2_updated.drop(["message", "good_embedding"], axis=1).to_csv("replaced_stuff_gigachat0.csv", index=False)

In [31]:
df2_updated.drop(["message", "good_embedding"], axis=1)

,solution_id,author_comment,author_comment_embedding
0,0,В вашем решении отсутствует использование f-ст...,-0.42553138732910156 -1.6270978450775146 -0.13...
1,1,Вы забыли поставить знак нижнего подчеркивания...,-0.1253896802663803 -1.1638524532318115 -0.086...
2,2,Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293266892433167 -1.477440595626831 0.2433...
3,3,"Обратите внимание, что в вашем коде ошибка в и...",-0.219484344124794 -1.6684648990631104 -0.0124...
4,4,В вашем решении ошибка в вычислении стоимости ...,-0.165065735578537 -1.1348521709442139 0.51365...
...,...,...,...
320,725,В вашем решении функция возвращает True в том ...,-0.5623469948768616 -1.0719929933547974 0.1257...
321,726,"Ваше решение возвращает список, тогда как согл...",-0.08087707310914993 -1.4105050563812256 0.423...
322,727,"Ваше решение возвращает список, но согласно ус...",0.02744465507566929 -1.2656797170639038 0.4701...
323,728,"В вашем решении нет синтаксических ошибок, но ...",-0.23759369552135468 -1.492260456085205 0.0606...


In [32]:
tests_info = pd.read_csv('/kaggle/input/ai-assistant-data/submit_finetunedYA-m0.2-v3.csv')

In [33]:
submission['tests_info'] = tests_info['author_comment']

In [34]:
def add_error_text(row):
    if "Ошибка в скрытых и открытых тестах.\n\n" in row['tests_info']:
        return "Ошибка в скрытых и открытых тестах.\n\n" + row['author_comment']
    else:
        return row['author_comment']

# Применение функции к каждой строке
submission['author_comment'] = submission.apply(add_error_text, axis=1)
submission['author_comment_embedding'] = submission['author_comment'].apply(lambda text: embedding2string(get_sentence_embedding(tokenizer, text)))

# two-model monster (did not succeed)

In [35]:
yagpt_output = pd.read_csv('/kaggle/input/ai-assistant-data/submit_finetunedYA-m0.2-v3.csv')

In [36]:
system_message=''' 
Ты - профессиональный программист и учитель.
Я пришлю тебе условие задачи, верное решение, мое решение с ошибками и ответ преподавателя. 
Дай крайне короткий ответ в одном предложении об ошибке в решении.
Ошибки могут быть синтаксические и смысловые.
Объясни ошибку через смысл задания. Ни в коем случае не ссылайся на реализацию в верном решении.
Ни в коем случае не исправленный присылай код решения.
Используй следующие фразы: "ваш код", "выполняет условия", "попробуйте изменить", "условия задания", "некорректно выполняет", "скорректировать ошибку" или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде. 
Обращайся ко мне лично и на вы.
'''

In [37]:
chat = GigaChat(credentials=pro_key, model='GigaChat-Pro', verify_ssl_certs=False) #GigaChat-Pro

messages = [
    SystemMessage(
        content=system_message
    )
]

model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [38]:
def predict(messages, tasks: pd.DataFrame, row_solution: pd.Series, synt_errors: str) -> str:
    text = f'''
    Условие задачи:
    {tasks[tasks['id'] == row_solution["task_id"]]['description'].values[0]}
    
    Мое решение с ошибкой: 
    {row_solution["student_solution"]}

    Ответ преподавателя:
    {tasks[tasks['id'] == row_solution["task_id"]]['author_answer'].values[0]}
    
    Старайся использовать только эти фразы: "ваш код", "выполняет условия", "попробуйте изменить", "условия задания", "некорректно выполняет", "скорректировать ошибку" или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде. 
    '''
    
    messages.append(HumanMessage(content=text))
    res = chat(messages)
    messages.append(res)
        
    return res.content

def get_sentence_embedding(tokenizer, sentence: str) -> torch.Tensor:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding

def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])

def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])

def remove_after_substring(text, substring='исправлен'):    
    index = text.lower().find(substring.lower())
    if index != -1:
        return text[:index]
    return text

def remove_code_from_text(text):
    code_pattern = re.compile(r'\b(def|class|if|else|elif|for|while|try|except|finally|with|import|from|print|return|break|continue|pass|raise|assert|yield|async|await|lambda|global|nonlocal|del|exec|eval)\b.*', re.DOTALL)
    fenced_code_pattern = re.compile(r'```.*?```', re.DOTALL)
    cleaned_text = code_pattern.sub('', text)
    cleaned_text = fenced_code_pattern.sub('', cleaned_text)
    return cleaned_text

def generate_submit(test_solutions_path: str, test_tasks_path: str, 
                    test_tests_path: str, syntacsis_erors_path:str, yagpt_output_path: str,
                    predict_func, save_path: str, use_tqdm: bool = True) -> None:
    test_solutions = pd.read_excel(test_solutions_path)
    test_tasks = pd.read_excel(test_tasks_path)
    test_tests = pd.read_excel(test_tests_path)
    with_syntacsis = pd.read_csv(syntacsis_erors_path)
    synt_errors_df = with_syntacsis[~with_syntacsis['message'].isna()]
    yagpt_output = pd.read_csv(yagpt_output_path)

    test_tasks['author_answer'] = yagpt_output['author_comment']
    
    bar = range(len(test_solutions))
    if use_tqdm:
        bar = tqdm.tqdm(bar, desc="Predicting")

    submit_df = pd.DataFrame(columns=["solution_id", "author_comment", "author_comment_embedding"])
    for i in bar:
        idx = test_solutions.index[i]
        solution_row = test_solutions.iloc[i]
        
        if i in synt_errors_df.index:
            synt_errors = synt_errors_df.loc[i]['message']
        else:
            synt_errors = 'не были найдены'
        
        try:
            chat = GigaChat(credentials=pro_key, model='GigaChat-Pro', verify_ssl_certs=False)
            messages = [
                SystemMessage(
                    content=system_message
                )
            ]
            text = predict_func(messages, test_tasks, solution_row, synt_errors)  # here you can do absolute whatever you want
            print(text)
            if 'исправлен' in text.lower():
                text = remove_after_substring(text)
            else:
                text = remove_code_from_text(text)
            
            embedding = embedding2string(get_sentence_embedding(tokenizer, text))
            submit_df.loc[i] = [idx, text, embedding]
        except:
            submit_df.to_csv(save_path, index=False)
            return i, submit_df
        submit_df.to_csv(save_path, index=False)

In [39]:
generate_submit(
    test_solutions_path="/kaggle/input/ai-assistant-data/solutions_test.xlsx",
    test_tasks_path="/kaggle/input/ai-assistant-data/tasks_test.xlsx",
    test_tests_path="/kaggle/input/ai-assistant-data/tests_test.xlsx",
    syntacsis_erors_path="/kaggle/input/ai-assistant-data/test_solutions_with_pyright.csv",
    yagpt_output_path = '/kaggle/input/ai-assistant-data/submit_finetunedYA-m0.2-v3.csv',
    predict_func=predict,
    save_path="submission_gigachat_wpop_words.csv",
    use_tqdm=True,
)

Predicting:  33%|███▎      | 1/3 [00:03<00:07,  3.84s/it]

Ваше решение некорректно выполняет условия задания, так как не использует f-строку для форматирования чисел в выводе.


Predicting:  67%|██████▋   | 2/3 [00:05<00:02,  2.73s/it]

Ваше решение некорректно выполняет условия задания, так как не использует f-строку для форматирования чисел в выводе.


Predicting: 100%|██████████| 3/3 [00:09<00:00,  3.05s/it]

Ваше решение некорректно выполняет условия задания, так как не использует f-строку для форматирования вывода.


## Check

In [40]:
ttmp = pd.read_csv('/kaggle/working/submission_gigachat_wpop_words.csv')
test_solutions = pd.read_excel("/kaggle/input/ai-assistant-data/solutions_test.xlsx")
with_syntacsis = pd.read_csv("/kaggle/input/ai-assistant-data/test_solutions_with_pyright.csv")
yagpt_output = pd.read_csv('/kaggle/input/ai-assistant-data/submit_finetunedYA-m0.2-v3.csv')

In [41]:
cnt = 0
for sol, error, comment, yagpt_answer in zip(
    test_solutions['student_solution'], 
    with_syntacsis['message'],
    ttmp['author_comment'],
    yagpt_output['author_comment']
):
    if cnt == 6:
        break
    print('STUDENT SOLUTION', sol)
    print('SYNTACSIS ERORRS', error)
    print('YAGPT ANSWER', yagpt_answer)
    print('GIGACHAT ANSWER', comment)
    print('_'*50)
    cnt += 1

STUDENT SOLUTION discount  = float(input())
money = int(input())

#ваш код ниже

print('Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.')
SYNTACSIS ERORRS nan
YAGPT ANSWER Ошибка в скрытых и открытых тестах.

 Вы забыли поставить f перед скобками.
GIGACHAT ANSWER Ваше решение некорректно выполняет условия задания, так как не использует f-строку для форматирования чисел в выводе.
__________________________________________________
STUDENT SOLUTION discount  = float(input())
money = int(input())

#ваш код ниже

print(f'Реализация проекта будет стоить {money} тыс. руб. без скидки. Со скидой стоимость составит {money- (money * discount)} тыс. руб.)
SYNTACSIS ERORRS "(" was not closed
YAGPT ANSWER Вы забыли поставить закрывающую кавычку в f-строке.
GIGACHAT ANSWER Ваше решение некорректно выполняет условия задания, так как не использует f-строку для форматирования чисел в выводе.
________________________________